## Model with arbitrary number of dealers:

There is $N$ dealers that that follow the rules described already for the two dealer model. The trhansaction occurs when $$\lvert p_i(t) - p_j(t) \rvert \ge L.$$

Transaction occurs only for the $i,j$ pair with largest price difference (which must be bigger than spread $L$). The remaining dealers switch their prices to the new market price ($p_k(t+1) = P_{n+1} = \frac{1}{2}(p_i(t) + p_j(t))$) with a probability $r$ and they keep their old prices with probability $1-r$. Here, $0\le r\le 1$  and $r$  is a constant equal for all dealers.

In [1]:
# Code for N dealers with switchin 

params={'N':100,'r':0.6,'L':0.01,'c':0.01,'dp':0.01,'dt':0.01**2, 'd':1.00, 'M':10} # define model parameters
p0 = np.ones(params['N'])*100.25

def model2(params,p0,numt):
     pswitch  = np.array([params['r'], 1.0-params['r']]) #switching rates 
    
    # compute running average Eq.(L6)
    def avgprice(dpn): 
        M = len(dpn)     
        weights  = np.array(range(1,M+1))*2.0/(M*(M+1))
        return weights.dot(dpn)
    
    # return transaction candidate: pair (i,j) with larget price difference
    def transactionPair(pt):  
        return [np.argmin(pt), np.argmax(pt)]

    # update prices for transaction between the two dealers (i,j) and  return new market price 
    def transactionUpdate(i,j,pt):
        newprice = np.average([pt[i], pt[j]])
        switch   = np.random.choice([True,False], p=pswitch,size=params['N']) # determine which dealers update price
        switch[i]=switch[j]=True
        pt[switch] = newprice
        return newprice
    
    # ... variable initializations ...
    mktprice = np.zeros(numt)   # initialize market price P(n)
    dmktprice= np.zeros(numt)   # initialize change in price dP(n) needed for running average
    ticktime = np.zeros(numt,dtype=np.int) #initialize array for tick times
    price    = p0               #initialize dealer's mid-price (p1,p2)
    time,tick= 0,0 # real time(t) and time time (n)
    deltapm  = 0.0 # trend term d <dP>_m dt for current random walk
    cdp      = params['c']*params['dp'] # define random step size
    ddt      = params['d']*params['dt'] # define amplitude of trend term
    
    while tick < numt: # loop over ticks
        [i,j]    = transactionPair(price)   # transaction candidate
        while np.abs(price[i]-price[j]) < params['L']: # transaction criterion Eq.(L1)
            price = price + deltapm + np.random.choice([-cdp,cdp], size=params['N']) # random walk step Eq.(L4)
            [i,j] = transactionPair(price) # update transaction candidate
            time += 1                      # update real time
        
        mktprice[tick] = transactionUpdate(i,j,price)
        
        # ... finalize loop ...
        dmktprice[tick]= mktprice[tick] - mktprice[np.max([0,tick-1])] # save dP(n) = P(n) - P(n-1)
        ticktime[tick] = time # save transaction time
        tick += 1             #update ticks
        tick0 = np.max([0, tick - params['M']])       #compute tick start for running average
        deltapm = avgprice(dmktprice[tick0:tick])*ddt #compute updated trend term for current tick time
    return ticktime,mktprice

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)

We can come up with yet refined model in that we let parameter $d_i$ to be different for every dealer and change in time:

$$p_i(t + \Delta t) = p_i(t) + d_i\langle\Delta P\rangle_M \Delta t + c f_i(t), \qquad i=1,2$$
$$d_i = \bar{d}(n) + \Delta d_i$$

where $\bar{d}$ is average trend-following effect (changing after each trade) and $\Delta d_i$, constant for each dealer, given by normal distribution with standard deviation $\sigma$.

The change in $\bar{d}$ is decribed as:

$$\bar{d}(n+1) = (1 - e_0) \bar{d}(n) + \phi(n)$$

where $e_0$ is constant ($e_0 \in [0 , 1]$) and $\phi(n)$ is independet Bernoulli distributed random noise:
$$\phi(n) = 
\begin{cases}
+0.01 & \text{probability 1/2} \\
-0.01 & \text{probability 1/2}
\end{cases}$$

In [ ]:
# Code for N dealers with switchin and params changig over ticks

params={'e0':1e-4,'sigma':0.1,'N':100,'r':0.6,'L':0.01,'c':0.01,'dp':0.01,'dt':0.01**2, 'd':1.00, 'M':10} # define model parameters
p0  = np.ones(params['N'])*100.25

def model2(params,p0,numt):
     pswitch  = np.array([params['r'], 1.0-params['r']]) #switching rates 
    
    # compute running average Eq.(L6)
    def avgprice(dpn): 
        M = len(dpn)     
        weights  = np.array(range(1,M+1))*2.0/(M*(M+1))
        return weights.dot(dpn)
    
    # return transaction candidate: pair (i,j) with larget price difference
    def transactionPair(pt):  
        return [np.argmin(pt), np.argmax(pt)]

    # update prices for transaction between the two dealers (i,j) and  return new market price 
    def transactionUpdate(i,j,pt):
        newprice = np.average([pt[i], pt[j]])
        switch   = np.random.choice([True,False], p=pswitch,size=params['N']) # determine which dealers update price
        switch[i]=switch[j]=True
        pt[switch] = newprice
        return newprice
    
    # ... variable initializations ...
    mktprice = np.zeros(numt)   # initialize market price P(n)
    dmktprice= np.zeros(numt)   # initialize change in price dP(n) needed for running average
    ticktime = np.zeros(numt,dtype=np.int) #initialize array for tick times
    price    = p0               #initialize dealer's mid-price (p1,p2)
    time,tick= 0,0 # real time(t) and time time (n)
    deltapm  = 0.0 # trend term d <dP>_m dt for current random walk
    dbar     = params['d']                                   # define average trend following effect \bar{d}
    di       = params['sigma']*np.random.randn(params['N'])  # trend following parameter for dealer i

    
    while tick < numt: # loop over ticks
        [i,j]    = transactionPair(price)   # transaction candidate
        while np.abs(price[i]-price[j]) < params['L']: # transaction criterion Eq.(L1)
            price = price + deltapm*(dbar+di) + np.random.choice([-cdp,cdp], size=params['N']) # random walk step Eq.(L4)
            [i,j] = transactionPair(price) # update transaction candidate
            time += 1                      # update real time
        
        dbar = (1.0 - params['e0'])*dbar + np.random.choice([0.01,-0.01])      
        mktprice[tick] = transactionUpdate(i,j,price)
        
        # ... finalize loop ...
        dmktprice[tick]= mktprice[tick] - mktprice[np.max([0,tick-1])] # save dP(n) = P(n) - P(n-1)
        ticktime[tick] = time # save transaction time
        tick += 1             #update ticks
        tick0 = np.max([0, tick - params['M']])       #compute tick start for running average
        deltapm = avgprice(dmktprice[tick0:tick])*params['dt'] # compute updated trend term for current tick time
    return ticktime,mktprice